# 학습

In [169]:
import pandas as pd
import numpy as np
from glob import glob

In [170]:
pd.set_option('display.max_rows', 500)

In [171]:
w_list = sorted(glob("competition_data/water_data/*.csv"))
r_list = sorted(glob("competition_data/rf_data/*.csv"))
sample=pd.read_csv('competition_data/sample_submission.csv')
total_wd=pd.DataFrame()
for i in range(0, len(w_list)):
    total_wd=pd.concat([total_wd, pd.read_csv(w_list[i])])
total_rf=pd.DataFrame()
for i in range(0, len(r_list)):
    total_rf=pd.concat([total_rf, pd.read_csv(r_list[i])])

In [172]:
df_rf=total_rf.copy()
df_water=total_wd.copy()
submission=sample.copy()

In [173]:
df_rf.set_index('ymdhm', inplace=True)
df_water.set_index('ymdhm', inplace=True)
submission.set_index('ymdhm', inplace=True)

In [174]:
df_rf.name = "rain_data"
df_water.name = "water_data"
submission.name = "submission"

In [175]:
def index_to_datetime(df,format):
    df.index = pd.to_datetime(df.index,
                              format=format)
    return df

In [176]:
df_rf=index_to_datetime(df=df_rf,format='%Y-%m-%d %H:%M')
df_water=index_to_datetime(df=df_water,format='%Y-%m-%d %H:%M')
submission=index_to_datetime(df=submission,format='%Y-%m-%d %H:%M')

In [177]:
df_rf.sort_index(inplace=True)
df_water.sort_index(inplace=True)
submission.sort_index(inplace=True)

In [178]:
# 데이터 시간대 확인하기
def check_datetime(df):
    print(df.name)
    print(df.select_dtypes('datetime64[ns]').head(1).index[0])
    print(df.select_dtypes('datetime64[ns]').tail(1).index[0])
    return None

check_datetime(df_rf)
check_datetime(df_water)
check_datetime(submission)

rain_data
2012-05-01 00:00:00
2022-07-18 23:50:00
water_data
2012-05-01 00:00:00
2022-07-18 23:50:00
submission
2022-06-01 00:00:00
2022-07-18 23:50:00


In [179]:
df_rf_copy=df_rf.copy()
df_water_copy=df_water.copy()

In [180]:
# data target 분리하기
target = df_water.loc[:,submission.columns]
data = pd.concat((df_rf,df_water.drop(submission.columns,axis=1)),axis=1)

In [181]:
# # data와 target 하나 밀어주기 (과거데이터를 사용해야 함으로)
_target = target.reset_index(drop=True)
_data = data.reset_index(drop=True)


In [182]:
_data.index+=1

In [183]:
tot=pd.concat((_data,_target),axis=1)

In [184]:
tot.sort_index(inplace=True)

In [185]:
df_rf_copy.reset_index(inplace=True)

In [186]:
df_datetime=df_rf_copy[['ymdhm']]

In [187]:
# tot 이랑 시간 붙혀주기
tot=pd.concat([df_datetime, tot], axis=1)

In [188]:
# 첫행이랑 막행 없앰 =nan있는 행 삭제
tot=tot.iloc[1:-1]

In [190]:
# 4. valid 로 넘어가기
total=tot.copy()

In [191]:
test=total[(total['ymdhm']>'2022-04-30')&(total['ymdhm']<'2022-06-01')]
test.reset_index(inplace=True, drop=True)

In [192]:
train=total[(total['ymdhm']<'2022-05-01')]
train.reset_index(inplace=True, drop=True)

In [193]:
# sklearn 랜포 다시 따라가는.
# 다시 분리
train_target = train.loc[:,submission.columns]
test_target = test.loc[:,submission.columns]

train_data = train.drop(submission.columns,axis=1)
test_data = test.drop(submission.columns,axis=1)
train_data=train_data.drop('ymdhm', axis=1)
test_data=test_data.drop('ymdhm', axis=1)

In [194]:
# null값 처리
train_target.fillna(train_target.mean(),inplace=True)
test_target.fillna(train_target.mean(),inplace=True)
train_data.fillna(train_data.mean(),inplace=True)
test_data.fillna(train_data.mean(),inplace=True)

In [195]:
print('--data--')
print(train_data.shape)
print(test_data.shape)
print('--target--')
print(train_target.shape)
print(test_target.shape)

--data--
(264959, 13)
(4464, 13)
--target--
(264959, 4)
(4464, 4)


In [196]:
from datetime import datetime 
import pandas_datareader.data as wb
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, SimpleRNN
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt

In [197]:
tmp=pd.DataFrame()
for i in range(test_target.shape[1]):
    #train/test size 설정
    train_size = int(len(train_data)*0.6)
    validation_size = int(len(train_data)*0.3)+train_size

    #train/test 학습 및 라벨 설정
    #종가를 예측하기 위해 종가를 label로 설정
    train_x = np.array(train_data[:train_size])
    train_y = np.array(train_target.iloc[:train_size,i])

    validation_x =np.array(train_data[train_size:validation_size])
    validation_y = np.array(train_target.iloc[train_size:validation_size, i])

    test_x = np.array(test_data)
    test_y = np.array(test_target.iloc[:,i])
    
    learning_rate = 0.01
    training_cnt = 20
    batch_size = 200
    input_size = train_x.shape[1]

    time_step = 1

    # reshape into (size(개수), time step, 입력 feature)
    train_x = train_x.reshape(train_x.shape[0],1,input_size)
    validation_x = validation_x.reshape(validation_x.shape[0], 1, input_size)
    test_x = test_x.reshape(test_x.shape[0], 1, input_size)
    

        # 모델 구조
    model = Sequential()
    model.add(LSTM(512,input_shape=(1,input_size))) # 512는 다른 숫자로도 가능
    model.add(Dropout(0.2)) 
    model.add(Dense(1,activation='relu')) #output(target)은 '종가'이기 때문에 1요소 = Dense의 output레이어는 1로 설정

    #오차 및 최적화기 설정
    model.compile(loss='mse',optimizer='rmsprop',metrics=['mae','mape'])
    model.summary()


    #학습
    history = model.fit(train_x,train_y,epochs=training_cnt, batch_size=batch_size, verbose=1)
    val_mse, val_mae, val_mape = model.evaluate(test_x, test_y, verbose=0)
    
    pred = model.predict(test_x)
    column_name='col_'+str(i)
    tmp[column_name]=pd.DataFrame(pred)
    model_name='model_'+str(i)
    model.save(model_name+'.h5')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 512)               1077248   
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 513       
                                                                 
Total params: 1,077,761
Trainable params: 1,077,761
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
795/795 [==============================] - 15s 17ms/step - loss: 43958.3203 - mae: 184.6959 - mape: 55.0764
Epoch 2/20
795/795 [==============================] - 13s 17ms/step - loss: 3545.0815 - mae: 32.1717 - mape: 8.6516
Epoch 3/20
795/795 [==============================] - 13s 17ms/step - los

795/795 [==============================] - 14s 18ms/step - loss: 294.7294 - mae: 11.0729 - mape: 3.5008
Epoch 10/20
795/795 [==============================] - 14s 18ms/step - loss: 288.3992 - mae: 11.0421 - mape: 3.4940
Epoch 11/20
795/795 [==============================] - 15s 19ms/step - loss: 279.2304 - mae: 10.8742 - mape: 3.4419
Epoch 12/20
795/795 [==============================] - 15s 18ms/step - loss: 276.6757 - mae: 10.8577 - mape: 3.4366
Epoch 13/20
795/795 [==============================] - 15s 18ms/step - loss: 276.6115 - mae: 10.9179 - mape: 3.4550
Epoch 14/20
795/795 [==============================] - 15s 19ms/step - loss: 270.4609 - mae: 10.7675 - mape: 3.4048
Epoch 15/20
795/795 [==============================] - 15s 18ms/step - loss: 270.5876 - mae: 10.7518 - mape: 3.4027
Epoch 16/20
795/795 [==============================] - 15s 19ms/step - loss: 263.6687 - mae: 10.6943 - mape: 3.3827
Epoch 17/20
795/795 [==============================] - 15s 19ms/step - loss: 269.547

# 우리들의 test

In [198]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [199]:
print('rmse : ', sqrt(mean_squared_error( test_target.values, tmp.values ))) 

rmse :  8.014679333570877


# 찐 test

In [228]:
real_test=total[total['ymdhm']>='2022-06-01']
real_test.reset_index(inplace=True, drop=True)
real_test = real_test.drop(submission.columns,axis=1)
real_test=real_test.drop('ymdhm', axis=1)
real_test.fillna(train_data.mean(),inplace=True)

In [230]:
real_test.shape

(6912, 13)

In [206]:
model_list=sorted(glob("*.h5"))

In [207]:
model_list

['model_0.h5', 'model_1.h5', 'model_2.h5', 'model_3.h5']

In [209]:
for i,j in enumerate(model_list):
    print(i,j)

0 model_0.h5
1 model_1.h5
2 model_2.h5
3 model_3.h5


In [212]:
from keras.models import load_model, save_model

In [231]:
for j,i in enumerate(model_list):
    test_x = np.array(real_test)
    test_x = test_x.reshape(test_x.shape[0], 1, test_x.shape[1])
    model=load_model(i)
    pred=model.predict(test_x)
    col=j+1
    sample.iloc[:, col]=pd.DataFrame(pred)

216/216 [==============================] - 1s 2ms/step


In [203]:
sample=pd.read_csv('competition_data/sample_submission.csv')

In [233]:
sample.to_csv('lstm_0818.csv', index=False)